In [1]:
import csv
%precision 4
with open("2019_pitches.csv") as fh:
    reader = csv.DictReader(fh)
    pitches = [row for row in reader]
  
pitches[0]

{'px': '0.0',
 'pz': '2.15',
 'start_speed': '88.8',
 'end_speed': '80.7',
 'spin_rate': 'placeholder',
 'spin_dir': 'placeholder',
 'break_angle': '22.8',
 'break_length': '4.8',
 'break_y': '24.0',
 'ax': '-8.47',
 'ay': '28.9',
 'az': '-15.51',
 'sz_bot': '1.7',
 'sz_top': '3.36',
 'type_confidence': 'placeholder',
 'vx0': '5.28',
 'vy0': '-128.95',
 'vz0': '-6.89',
 'x': '116.97',
 'x0': '-1.42',
 'y': '180.81',
 'y0': '50.0',
 'z0': '6.07',
 'pfx_x': '-5.07',
 'pfx_z': '9.98',
 'nasty': '',
 'zone': 'placeholder',
 'code': 'X',
 'type': 'X',
 'pitch_type': 'FF',
 'event_num': '5',
 'b_score': '0.0',
 'ab_id': '2019000001.0',
 'b_count': '0.0',
 's_count': '0.0',
 'outs': '0.0',
 'pitch_num': '1.0',
 'on_1b': '0.0',
 'on_2b': '0.0',
 'on_3b': '0.0'}

In [2]:
pitches[2]

{'px': '-0.05',
 'pz': '2.03',
 'start_speed': '85.7',
 'end_speed': '79.6',
 'spin_rate': 'placeholder',
 'spin_dir': 'placeholder',
 'break_angle': '9.6',
 'break_length': '6.0',
 'break_y': '24.0',
 'ax': '3.65',
 'ay': '22.07',
 'az': '-22.64',
 'sz_bot': '1.59',
 'sz_top': '3.55',
 'type_confidence': 'placeholder',
 'vx0': '2.33',
 'vy0': '-124.6',
 'vz0': '-5.98',
 'x': '118.86',
 'x0': '-1.29',
 'y': '183.96',
 'y0': '50.0',
 'z0': '6.3',
 'pfx_x': '2.3',
 'pfx_z': '5.99',
 'nasty': '',
 'zone': 'placeholder',
 'code': 'S',
 'type': 'S',
 'pitch_type': 'SL',
 'event_num': '9',
 'b_score': '0.0',
 'ab_id': '2019000002.0',
 'b_count': '0.0',
 's_count': '0.0',
 'outs': '1.0',
 'pitch_num': '2.0',
 'on_1b': '0.0',
 'on_2b': '0.0',
 'on_3b': '0.0'}

In [3]:
# Pitcher L, R can be detected by looking at x0. If x0 < 0 == R, else L
# We could also use vx0. If vx0 > 0 == R, else L (opposite sign of x0)
for row in pitches:
    if float(row["x0"]) > 0:
        print(row)
        break

{'px': '0.37', 'pz': '2.07', 'start_speed': '90.5', 'end_speed': '83.2', 'spin_rate': 'placeholder', 'spin_dir': 'placeholder', 'break_angle': '27.6', 'break_length': '4.8', 'break_y': '24.0', 'ax': '11.97', 'ay': '26.73', 'az': '-19.0', 'sz_bot': '1.72', 'sz_top': '3.66', 'type_confidence': 'placeholder', 'vx0': '-7.47', 'vy0': '-131.53', 'vz0': '-6.28', 'x': '102.82', 'x0': '2.36', 'y': '182.94', 'y0': '50.0', 'z0': '5.88', 'pfx_x': '6.8', 'pfx_z': '7.49', 'nasty': '', 'zone': 'placeholder', 'code': 'C', 'type': 'C', 'pitch_type': 'FF', 'event_num': '5', 'b_score': '0.0', 'ab_id': '2019000066.0', 'b_count': '0.0', 's_count': '0.0', 'outs': '0.0', 'pitch_num': '1.0', 'on_1b': '0.0', 'on_2b': '0.0', 'on_3b': '0.0'}


In [4]:
# Speed range by pitch type
# quantiles is the data

from statistics import quantiles
from collections import defaultdict

pitch_types = ["FF", "FT", "SL", "SI", "CH", "CU", "FC", "KC", "FS"]
    
def get_custom(getter, by_pitch_type=True, by_handedness=False):
    stat = defaultdict(list)
    for p in pitches:
        p_type = p["pitch_type"]
        if p_type not in pitch_types:
            continue

        if not by_pitch_type:
            p_type = "ALL"
        if by_handedness:
            handed = "R" if float(p["vx0"]) > 0 else "L"
            p_type += "_" + handed
        stat[p_type].append(getter(p))

    return {k: quantiles(v) for k, v in stat.items()}

def get_quartiles(property: str, by_pitch_type=True, by_handedness=False):
    """Aggregate the value of the property and return quartiles"""
    return get_custom(lambda x: float(x[property]), by_pitch_type, by_handedness)

speed_start = get_quartiles("start_speed")
speed_end = get_quartiles("end_speed")

for p_type in speed_start.keys():
    print(f"{p_type}: {speed_start[p_type]}   {speed_end[p_type]}")

FF: [91.8, 93.5, 95.2]   [84.3, 85.9, 87.4]
SL: [82.6, 84.8, 86.9]   [76.4, 78.6, 80.6]
CH: [82.5, 84.8, 86.9]   [76.1, 78.3, 80.2]
FT: [91.0, 92.8, 94.5]   [83.7, 85.4, 86.9]
SI: [90.2, 92.3, 94.1]   [83.0, 84.8, 86.6]
KC: [79.1, 80.8, 82.5]   [72.9, 74.7, 76.3]
CU: [76.2, 78.8, 81.1]   [70.4, 72.8, 75.0]
FC: [86.6, 88.3, 90.4]   [80.2, 81.8, 83.7]
FS: [83.6, 85.3, 86.9]   [77.2, 78.9, 80.4]


In [5]:
# counts by pitch type

total_count = 0.0
counts = defaultdict(int)
for p in pitches:
    p_type = p["pitch_type"]
    if p_type not in pitch_types:
        continue

    counts[p_type] += 1
    total_count += 1

for p_type, count in counts.items():
    print(f"{p_type}: {100*count/total_count:.4}%")

FF: 36.22%
SL: 17.69%
CH: 11.07%
FT: 8.456%
SI: 7.669%
KC: 2.123%
CU: 8.889%
FC: 6.419%
FS: 1.46%


In [6]:
# p_x and p_y is where a pitch crosses the plate
# What is pfx_x and pfx_z

In [7]:
# Conclusion: 
pfx_x = get_quartiles("pfx_x", by_handedness=True)
pfx_z = get_quartiles("pfx_z")

def print_by_handedness(quarts):
    for p_type in pitch_types:
        R = p_type + "_R"
        L = p_type + "_L"
        print(f"{p_type} - R: {quarts[R]}\t L: {quarts[L]}")
        
print_by_handedness(pfx_x)
pfx_z

FF - R: [-6.36, -4.84, -3.29]	 L: [3.02, 4.81, 6.45]
FT - R: [-9.98, -8.93, -7.75]	 L: [7.75, 8.87, 9.86]
SL - R: [0.59, 1.93, 3.48]	 L: [-3.25, -1.54, -0.05]
SI - R: [-10.28, -9.19, -8.0]	 L: [7.42, 8.62, 9.79]
CH - R: [-9.57, -8.32, -6.84]	 L: [7.02, 8.55, 9.96]
CU - R: [2.4, 4.46, 6.46]	 L: [-5.47, -3.08, 0.99]
FC - R: [-0.19, 1.05, 2.16]	 L: [-0.97, -0.01, 1.09]
KC - R: [2.02, 3.93, 5.795]	 L: [-2.83, -0.76, 1.24]
FS - R: [-7.915, -6.36, -4.73]	 L: [-3.69, -1.27, 1.44]


{'FF': [8.0400, 9.1900, 10.2200],
 'SL': [-0.4200, 1.3200, 2.9600],
 'CH': [2.4800, 4.3900, 6.0900],
 'FT': [4.6100, 6.0600, 7.4200],
 'SI': [3.4000, 5.1700, 6.7950],
 'KC': [-7.9100, -6.1200, -4.0300],
 'CU': [-7.3500, -5.1600, -2.7000],
 'FC': [3.2200, 4.6100, 6.0300],
 'FS': [1.1100, 2.6900, 4.5500]}

In [16]:
from math import sqrt
pitch = pitches[2]
pitch


# vy0*t + 0.5*ay*t^2 = -50
# 0.5*ay*t^2 + vy0*t + 50 = 0
# t = [-vy0 +- sqrt(vy0^2 - 2*ay*50)]/(ay)
def get_duration(pitch):
    vy0 = float(pitch['vy0'])
    ay = float(pitch['ay'])
    t = (-vy0 - sqrt(vy0*vy0 - (ay*2*50)))/ay
    return t

get_custom(get_duration)

{'FF': [0.3765, 0.3833, 0.3903],
 'SL': [0.4116, 0.4219, 0.4337],
 'CH': [0.4123, 0.4222, 0.4343],
 'FT': [0.3793, 0.3859, 0.3937],
 'SI': [0.3807, 0.3884, 0.3973],
 'KC': [0.4352, 0.4445, 0.4544],
 'CU': [0.4426, 0.4557, 0.4718],
 'FC': [0.3953, 0.4046, 0.4127],
 'FS': [0.4121, 0.4198, 0.4285]}

In [17]:
ax = float(pitch['ax'])
vx0 = float(pitch['vx0'])
x0 = float(pitch['x0'])
px = float(pitch['px'])
pfx_x = float(pitch['pfx_x'])

# how much did the ball break?
t = get_duration(pitch)
simple_expected_x = vx0 * t

x0 + vx0*t + 0.5*ax*t*t, pitch # -0.002 vs -0.05
0.5*ax*t*t, pitch # 1.29 vs 2.33

# Let's see how our vt+0.5*a*t*t compares with recorded positions

def get_expected_location_error(pitch):
    ax = float(pitch['ax'])
    vx0 = float(pitch['vx0'])
    x0 = float(pitch['x0'])
    px = float(pitch['px'])
    pfx_x = float(pitch['pfx_x'])
    t = get_duration(pitch)

    return x0 + vx0*t + 0.5*ax*t*t - px

get_custom(get_expected_location_error, False)

# Conclusion: So the physics model connects well with the final px values.
# pfx_x doesn't seem to make sense but that's okay, we don't need it.

{'ALL': [-0.0189, 0.0202, 0.0563]}

In [18]:

def get_break_x(pitch):
    ax = float(pitch['ax'])
    t = get_duration(pitch)
    return 0.5*ax*t*t*12

x_breaks_inches = get_custom(get_break_x, by_handedness=True)
print_by_handedness(x_breaks_inches)

FF - R: [-10.52954458030921, -8.017573039688758, -5.444577072447688]	 L: [5.012258101644899, 7.961908129858508, 10.679485750599781]
FT - R: [-16.521161627843778, -14.80098590428748, -12.843060948509963]	 L: [12.837341055597753, 14.691911548056062, 16.328799653315293]
SL - R: [0.9813536220371526, 3.2054172772907568, 5.764469697571114]	 L: [-5.38310928245169, -2.555653928559615, -0.07856020515474918]
SI - R: [-17.02658293066584, -15.228463983224433, -13.256339808690782]	 L: [12.292954034981145, 14.274589081123104, 16.21849508071324]
CH - R: [-15.845849711146661, -13.779414939566905, -11.342028200242416]	 L: [11.638850832762698, 14.167734675733321, 16.48893081254913]
CU - R: [3.9715930426072985, 7.379820729473471, 10.689031165991334]	 L: [-9.046520574482745, -5.094323783844151, 1.6309220449947732]
FC - R: [-0.32292631728745436, 1.7430096227248575, 3.577500609524885]	 L: [-1.6138590898915661, -0.010634590551029279, 1.8113834982392736]
KC - R: [3.348335277418868, 6.510617848305445, 9.582041

In [19]:
# the calculated x_break seems very high for fastballs. What does it look like in pfx_x?

print_by_handedness(get_quartiles("pfx_x", by_handedness=True))

FF - R: [-6.36, -4.84, -3.29]	 L: [3.02, 4.81, 6.45]
FT - R: [-9.98, -8.93, -7.75]	 L: [7.75, 8.87, 9.86]
SL - R: [0.59, 1.93, 3.48]	 L: [-3.25, -1.54, -0.05]
SI - R: [-10.28, -9.19, -8.0]	 L: [7.42, 8.62, 9.79]
CH - R: [-9.57, -8.32, -6.84]	 L: [7.02, 8.55, 9.96]
CU - R: [2.4, 4.46, 6.46]	 L: [-5.47, -3.08, 0.99]
FC - R: [-0.19, 1.05, 2.16]	 L: [-0.97, -0.01, 1.09]
KC - R: [2.02, 3.93, 5.795]	 L: [-2.83, -0.76, 1.24]
FS - R: [-7.915, -6.36, -4.73]	 L: [-3.69, -1.27, 1.44]


# Discussion

## Duration and full displacement

`get_expected_location_error` is reporting pretty small numbers
so that gives me confidence in our `t` calculation. Additionally
it also seems right that our view of x + v*t + 0.5*a*t*t is the right
way to figure out where the ball is.

## Breaking distance, acceleration change vs pfx

We would expect `pfx = 0.5*a*t*t`
but it seems that `0.5*a*t*t > pfx`

So some documentation says pfx is the movement in in inches in the last 40ft. Our x_breaks_inches calculation is for 50ft
so it makes sense that it is larger. Is it only 20% larger? It should actually be less than 20% since the acceleration should
have even less effect in the early part of the journey

Some other documentation says that pfx is in feet but that is just insane. Unless it means ALL of the movement? I checked it but that doesn't seem likely either

In [36]:
# Maybe we can start looking at z change and that will give us a better idea
pfx_z = get_quartiles("pfx_z")

def get_break_z(pitch):
    az = float(pitch['az'])
    t = get_duration(pitch)
    return 0.5*az*t*t*12

def get_z_travel(pitch):
    az = float(pitch['az'])
    t = get_duration(pitch)
    vz0 = float(pitch['vz0'])
    return vz0*t + 0.5*az*t*t

def get_confirmed_z(pitch):
    pz = float(pitch['pz'])
    z0 = float(pitch['z0'])
    return pz - z0

pfx_z, get_custom(get_break_z)

({'FF': [8.0400, 9.1900, 10.2200],
  'SL': [-0.4200, 1.3200, 2.9600],
  'CH': [2.4800, 4.3900, 6.0900],
  'FT': [4.6100, 6.0600, 7.4200],
  'SI': [3.4000, 5.1700, 6.7950],
  'KC': [-7.9100, -6.1200, -4.0300],
  'CU': [-7.3500, -5.1600, -2.7000],
  'FC': [3.2200, 4.6100, 6.0300],
  'FS': [1.1100, 2.6900, 4.5500]},
 {'FF': [-15.3193, -13.2146, -11.2695],
  'SL': [-36.4584, -32.3558, -28.4823],
  'CH': [-30.8819, -27.4686, -24.2798],
  'FT': [-21.5692, -18.8820, -16.2357],
  'SI': [-23.7316, -20.4215, -17.4646],
  'KC': [-52.3889, -48.7223, -43.8214],
  'CU': [-53.9961, -48.6558, -43.5023],
  'FC': [-26.8620, -23.8002, -20.8096],
  'FS': [-32.6265, -29.4994, -26.2584]})

In [59]:
# Get z error

def get_z_expected_location_error(pitch):
    az = float(pitch['az'])
    vz0 = float(pitch['vz0'])
    z0 = float(pitch['z0'])
    pz = float(pitch['pz'])
    pfx_z = float(pitch['pfx_z'])
    t = get_duration(pitch)

    return (z0 + vz0*t + 0.5*az*t*t - pz)*12

def get_z_error_by_pfx_z(pitch):
    az = float(pitch['az'])
    vz0 = float(pitch['vz0'])
    z0 = float(pitch['z0'])
    pz = float(pitch['pz'])
    pfx_z = float(pitch['pfx_z'])
    t = get_duration(pitch)
    return (z0 + pfx_z - 0.5*32*t*t + vz0*t - pz)*12

get_custom(get_z_expected_location_error, False), get_custom(get_z_error_by_pfx_z, False)

({'ALL': [-2.3818, -1.9305, -1.5687]}, {'ALL': [15.4968, 55.7170, 88.0237]})

# PFX_? is useless

Maybe I am interpreting them incorrectly but they are way off.

# Path forward

We are going to ignore it and go with the normal 9 point fit: x0, y0, z0, vx0, vy0, vz0, ax, ay, az




In [62]:
start_x = get_quartiles('x0', by_handedness=True, by_pitch_type=False)
start_z = get_quartiles('z0', by_handedness=True, by_pitch_type=False)

start_x, start_z

({'ALL_R': [-2.1500, -1.6900, -1.2600], 'ALL_L': [1.4000, 1.9100, 2.3800]},
 {'ALL_R': [5.5000, 5.7800, 6.0600], 'ALL_L': [5.5400, 5.8100, 6.1000]})

In [63]:
get_quartiles('vy0')

{'FF': [-138.3300, -135.8800, -133.4300],
 'SL': [-126.3600, -123.3600, -120.1300],
 'CH': [-126.2900, -123.3100, -119.9200],
 'FT': [-137.3000, -134.9200, -132.3000],
 'SI': [-136.7400, -134.1100, -131.0800],
 'KC': [-119.9275, -117.5400, -115.0825],
 'CU': [-117.9200, -114.6300, -110.8200],
 'FC': [-131.4300, -128.4100, -125.9300],
 'FS': [-126.2700, -124.0100, -121.5600]}

In [85]:

# copied from above
def get_break_x(pitch):
    ax = float(pitch['ax'])
    t = get_duration(pitch)
    break_inches = 0.5*ax*t*t*12
    pretty = round(1000*break_inches)/1000
    return pretty

x_breaks_inches = get_custom(get_break_x, by_handedness=True)
print_by_handedness(x_breaks_inches)


def get_csharp_type(ptype):
    names = {
        "FF": "FourFastball",
        "FT": "TwoFastball",
        "SL": "Slider",
        "SI": "Sinker",
        "CH": "Changeup",
        "CU": "Curveball",
        "FC": "Cutter",
        "KC": "KnuckleCurve",
        "FS": "Splitter"
    }
    return names[ptype]

def csharp_helper(quarts, by_handedness=True):
    output = "{\n"
    for p_type in pitch_types:
        row = None
        if by_handedness:
            R = p_type + "_R"
            L = p_type + "_L"
            row = quarts[L]
        else:
            row = quarts[p_type]
        output += "{ PitchType." + get_csharp_type(p_type) + ", new MinMidMax("

        output += f"{row[0]}f, {row[1]}f, {row[2]}f)"
        output += "},\n"
    output += "}\n"
    print(output)

csharp_helper(x_breaks_inches)

FF - R: [-10.53, -8.018, -5.445]	 L: [5.012, 7.962, 10.6795]
FT - R: [-16.521, -14.801, -12.843]	 L: [12.837, 14.692, 16.329]
SL - R: [0.981, 3.205, 5.7642500000000005]	 L: [-5.383, -2.5555000000000003, -0.0785]
SI - R: [-17.02675, -15.2285, -13.25625]	 L: [12.293, 14.275, 16.2185]
CH - R: [-15.846, -13.779, -11.342]	 L: [11.639, 14.168, 16.489]
CU - R: [3.972, 7.38, 10.689]	 L: [-9.04625, -5.0945, 1.6312499999999999]
FC - R: [-0.323, 1.743, 3.578]	 L: [-1.61375, -0.010499999999999999, 1.81175]
KC - R: [3.3485, 6.511, 9.582]	 L: [-4.694, -1.263, 2.051]
FS - R: [-13.1175, -10.544, -7.845000000000001]	 L: [-6.1205, -2.106, 2.3775]
{
{ PitchType.FourFastball, new MinMidMax(5.012f, 7.962f, 10.6795f)},
{ PitchType.TwoFastball, new MinMidMax(12.837f, 14.692f, 16.329f)},
{ PitchType.Slider, new MinMidMax(-5.383f, -2.5555000000000003f, -0.0785f)},
{ PitchType.Sinker, new MinMidMax(12.293f, 14.275f, 16.2185f)},
{ PitchType.Changeup, new MinMidMax(11.639f, 14.168f, 16.489f)},
{ PitchType.Curveba

In [86]:
def get_break_z(pitch):
    # includes gravity break
    az = float(pitch['az'])
    t = get_duration(pitch)
    break_inches = 0.5*az*t*t*12
    pretty = round(1000*break_inches)/1000
    return pretty

csharp_helper(get_custom(get_break_z), by_handedness=False)

{
{ PitchType.FourFastball, new MinMidMax(-15.319f, -13.215f, -11.27f)},
{ PitchType.TwoFastball, new MinMidMax(-21.569f, -18.882f, -16.236f)},
{ PitchType.Slider, new MinMidMax(-36.458f, -32.356f, -28.482f)},
{ PitchType.Sinker, new MinMidMax(-23.732f, -20.422f, -17.465f)},
{ PitchType.Changeup, new MinMidMax(-30.882f, -27.469f, -24.28f)},
{ PitchType.Curveball, new MinMidMax(-53.996f, -48.656f, -43.502f)},
{ PitchType.Cutter, new MinMidMax(-26.862f, -23.8f, -20.81f)},
{ PitchType.KnuckleCurve, new MinMidMax(-52.389250000000004f, -48.722f, -43.82125f)},
{ PitchType.Splitter, new MinMidMax(-32.62625f, -29.499499999999998f, -26.25875f)},
}



In [88]:
csharp_helper(get_quartiles("ay"), by_handedness=False)

{
{ PitchType.FourFastball, new MinMidMax(26.53f, 28.29f, 30.05f)},
{ PitchType.TwoFastball, new MinMidMax(26.13f, 27.81f, 29.52f)},
{ PitchType.Slider, new MinMidMax(21.38f, 22.9f, 24.52f)},
{ PitchType.Sinker, new MinMidMax(25.75f, 27.51f, 29.31f)},
{ PitchType.Changeup, new MinMidMax(21.4f, 23.11f, 24.83f)},
{ PitchType.Curveball, new MinMidMax(19.73f, 21.33f, 23.0f)},
{ PitchType.Cutter, new MinMidMax(22.44f, 23.96f, 25.61f)},
{ PitchType.KnuckleCurve, new MinMidMax(21.01f, 22.46f, 23.95f)},
{ PitchType.Splitter, new MinMidMax(21.91f, 23.28f, 24.64f)},
}



# Gather conclusions

# Starting location by handedness

Starting x:

```
{'ALL_R': [-2.1500, -1.6900, -1.2600], 'ALL_L': [1.4000, 1.9100, 2.3800]}
```

Starting y: 

```
50ft
```

Starting z:

```
 {'ALL_R': [5.5000, 5.7800, 6.0600], 'ALL_L': [5.5400, 5.8100, 6.1000]}
```

# vy0 by pitch type (feet per second)

```
{'FF': [-138.3300, -135.8800, -133.4300],
 'SL': [-126.3600, -123.3600, -120.1300],
 'CH': [-126.2900, -123.3100, -119.9200],
 'FT': [-137.3000, -134.9200, -132.3000],
 'SI': [-136.7400, -134.1100, -131.0800],
 'KC': [-119.9275, -117.5400, -115.0825],
 'CU': [-117.9200, -114.6300, -110.8200],
 'FC': [-131.4300, -128.4100, -125.9300],
 'FS': [-126.2700, -124.0100, -121.5600]}
```

# Break distance in x by pitch type and handedness (inches)

```
FF - R: [-10.53, -8.018, -5.445]	 L: [5.012, 7.962, 10.6795]
FT - R: [-16.521, -14.801, -12.843]	 L: [12.837, 14.692, 16.329]
SL - R: [0.981, 3.205, 5.76425]      L: [-5.383, -2.5555, -0.0785]
SI - R: [-17.03, -15.23, -13.26]	 L: [12.293, 14.275, 16.2185]
CH - R: [-15.846, -13.779, -11.342]	 L: [11.639, 14.168, 16.489]
CU - R: [3.972, 7.38, 10.689]	     L: [-9.04625, -5.0945, 1.63125]
FC - R: [-0.323, 1.743, 3.578]	     L: [-1.61375, -0.0105, 1.81175]
KC - R: [3.3485, 6.511, 9.582]	     L: [-4.694, -1.263, 2.051]
FS - R: [-13.1175, -10.544, -7.845]	 L: [-6.1205, -2.106, 2.3775]
```

# Break distance in z by pitch type (inches)

This includes the gravity Break

```
{'FF': [-15.3190, -13.2150, -11.2700],
 'SL': [-36.4580, -32.3560, -28.4820],
 'CH': [-30.8820, -27.4690, -24.2800],
 'FT': [-21.5690, -18.8820, -16.2360],
 'SI': [-23.7320, -20.4220, -17.4650],
 'KC': [-52.3893, -48.7220, -43.8212],
 'CU': [-53.9960, -48.6560, -43.5020],
 'FC': [-26.8620, -23.8000, -20.8100],
 'FS': [-32.6262, -29.4995, -26.2587]}
```

# How to use these values:

We will have an algorithm that picks end location and pitch time.

Starting location: just get average starting location by handedness

Pick vy0 by pitch type and pick average ay. This gives us `t`

Now get average break distance in x and z by pitch type and handedness

We will calculate `ax` and `az` such that `0.5*a*t*t` equalts the break distance

Now take the target location, subtract the break distances to get straight_x and straight_z

Use these to set vx0 and vz0.